# Wiki-40Bデータセットを構造化する


## ライブラリのインストール

In [1]:
!pip install -q tensorflow-datasets

## 設定

In [2]:
LANGUAGE = "ja"
WIKI40B_VERSION = "1.3.0"

## Wiki-40Bデータセットのダウンロード

ダウンロードしたデータセットをファイルに保存する。

In [3]:
import tensorflow_datasets as tfds

ds, ds_info = tfds.load(name=f"wiki40b/{LANGUAGE}", 
          shuffle_files=False,
          with_info=True)

print(ds_info)

tfds.core.DatasetInfo(
    name='wiki40b',
    version=1.3.0,
    description='Clean-up text for 40+ Wikipedia languages editions of pages
correspond to entities. The datasets have train/dev/test splits per language.
The dataset is cleaned up by page filtering to remove disambiguation pages,
redirect pages, deleted pages, and non-entity pages. Each example contains the
wikidata id of the entity, and the full Wikipedia article after page processing
that removes non-content sections and structured objects. The language models
trained on this corpus - including 41 monolingual models, and 2 multilingual
models - can be found at https://tfhub.dev/google/collections/wiki40b-lm/1.',
    homepage='https://research.google/pubs/pub49029/',
    features=FeaturesDict({
        'text': Text(shape=(), dtype=tf.string),
        'version_id': Text(shape=(), dtype=tf.string),
        'wikidata_id': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=828236,
    splits={
        'test': 4126

In [4]:
# 表示してみる

for i, datum in enumerate(ds["test"].as_numpy_iterator()):
    text = datum["text"].decode("utf-8")
    version_id = datum["version_id"].decode("utf-8")
    wikidata_id = datum["wikidata_id"].decode("utf-8")

    print(f"{i}. version_id: {version_id}, wikidata_id: {wikidata_id}, text: {text}")
    print("------------------------------------------------")
    print()

    if i >= 2:
        break

0. version_id: 1848243370795951995, wikidata_id: Q11331136, text: 
_START_ARTICLE_
ビートたけしの教科書に載らない日本人の謎
_START_SECTION_
概要
_START_PARAGRAPH_
「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。_NEWLINE_新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。
------------------------------------------------

1. version_id: 13993402720669107168, wikidata_id: Q17218581, text: 
_START_ARTICLE_
ゲオネットワークス
_START_SECTION_
概要
_START_PARAGRAPH_
ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。_NEWLINE_2005年に株式会社ライブドアから分割されて設立。_NEWLINE_かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。_NEWLINE_「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。_NEWLINE_オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。
_START_SECTION_
ラジオCM
_ST

In [5]:
import json
from tqdm import tqdm
import tensorflow_datasets as tfds
import gzip

for split_name in ["test", "validation", "train"]:
    ds, ds_info = tfds.load(f"wiki40b/{LANGUAGE}:{WIKI40B_VERSION}", 
        shuffle_files=False,
        with_info=True)

    with gzip.open(f"wiki40b_{LANGUAGE}_{WIKI40B_VERSION}_{split_name}.jsonl.gz", "wt", encoding="utf-8") as f_out:
        for datum in tqdm(ds[split_name].as_numpy_iterator(), total=ds_info.splits[split_name].num_examples):
            text = datum["text"].decode("utf-8")
            version_id = datum["version_id"].decode("utf-8")
            wikidata_id = datum["wikidata_id"].decode("utf-8")

            f_out.write(json.dumps({
                "version_id": version_id, 
                "wikidata_id": wikidata_id, 
                "text": text
                }, ensure_ascii=True))
            f_out.write("\n")

100%|██████████| 745392/745392 [35:14<00:00, 352.47it/s]


## Wiki-40Bデータセットの構造解析

Wiki-40Bデータセットからタイトル、セクション、パラグラフの情報を抽出する。  

注意:次のデータは読み捨てている。

- タイトルやセクション名に改行文字が入ってる場合、その文書全体を無視する。
- セクション名だけでパラグラフがない場合、そのセクションを無視する。
- その他、コード中のアサーションに違反している場合、その文書全体を無視する。

In [7]:
import json
import gzip
import enum
import unicodedata
import re


def normalize_ja_text(text):
    # 必要に応じて正規化処理を施す。この例では何もしない。
    return text


class ParseMode(enum.Enum):
    INIT = enum.auto()
    IN_TITLE = enum.auto()
    IN_SECTION = enum.auto()
    IN_PARAGRAPH = enum.auto()
    EXPECT_SECTION = enum.auto()
    EXPECT_PARAGRAPH = enum.auto()


def extract_content(text, normalizer):
    title = None
    sections = []

    def normalize_line(line):
        assert "\n" not in line
        lines = line.split("_NEWLINE_")
        line = normalizer("_NEWLINE_").join([normalizer(l.strip()) for l in lines])
        return line

    mode = ParseMode.INIT

    for line in text.split("\n")[1:]:
        line = line.strip()
        if mode == ParseMode.INIT:
            assert line == "_START_ARTICLE_", line
            mode = ParseMode.IN_TITLE
        elif mode == ParseMode.IN_TITLE:
            title = normalize_line(line)
            mode = ParseMode.EXPECT_SECTION
        elif mode == ParseMode.IN_SECTION:
            sections.append({"name": normalize_line(line), "paragraph": None})
            mode = ParseMode.EXPECT_PARAGRAPH
        elif mode == ParseMode.IN_PARAGRAPH:
            assert len(sections) > 0, line
            last_section = sections[-1]
            assert last_section["paragraph"] is None, line
            last_section["paragraph"] = normalize_line(line)
            mode = ParseMode.EXPECT_SECTION
        elif mode == ParseMode.EXPECT_SECTION:
            assert line == "_START_SECTION_" or line == "_START_PARAGRAPH_", line
            if line == "_START_SECTION_":
                mode = ParseMode.IN_SECTION
            elif line == "_START_PARAGRAPH_":
                sections.append({"name": None, "paragraph": None})
                mode = ParseMode.IN_PARAGRAPH
        elif mode == ParseMode.EXPECT_PARAGRAPH:
            assert line == "_START_PARAGRAPH_", line
            mode = ParseMode.IN_PARAGRAPH
        
    for section in sections:
        assert section["paragraph"] is not None, text

    sections = [section for section in sections if len(section["paragraph"]) > 0]  # パラグラフのないセクションは無視する。

    return title, sections


def parse_wiki40b(language, wiki40b_version, split_name, normalizer):
    with gzip.open(f"wiki40b_{language}_{wiki40b_version}_{split_name}.jsonl.gz", "rt", encoding="utf-8") as f_in, \
        gzip.open(f"wiki40b_{language}_{wiki40b_version}_{split_name}.parsed.jsonl.gz", "wt", encoding="utf-8") as f_parsed:

        for line in f_in:
            page_data = json.loads(line)

            version_id = page_data["version_id"]
            wikidata_id = page_data["wikidata_id"]
            text = page_data["text"]
            try:
                title, sections = extract_content(text, normalizer)
            except AssertionError as err:
                # アサーションエラーがあった場合は処理をスキップする。
                print(f"WARN: {err} in {wikidata_id}")
                # raise err
                continue

            assert title is not None
            normalized_text = normalizer("_START_ARTICLE_") + "\n"
            normalized_text += title + "\n"

            for section in sections:
                name = section["name"]
                paragraph = section["paragraph"]
                if name is not None:
                    normalized_text += normalizer("_START_SECTION_") + "\n"
                    normalized_text += name + "\n"

                assert paragraph is not None
                normalized_text += normalizer("_START_PARAGRAPH_") + "\n"
                normalized_text += paragraph + "\n"

            page_data["normalized_title"] = title
            page_data["normalized_sections"] = sections
            page_data["normalized_text"] = normalized_text
            
            f_parsed.write(json.dumps(page_data, ensure_ascii=False) + "\n")

In [14]:
parse_wiki40b(LANGUAGE, WIKI40B_VERSION, "test",  normalize_ja_text)
parse_wiki40b(LANGUAGE, WIKI40B_VERSION, "validation", normalize_ja_text)
parse_wiki40b(LANGUAGE, WIKI40B_VERSION, "train", normalize_ja_text)

WARN:  in Q1916324
WARN:  in Q56354326
WARN:  in Q5015732
WARN:  in Q2005236
WARN:  in Q1158767


## 構造解析した結果の表示

In [9]:
import json
import gzip


with gzip.open(f"wiki40b_{LANGUAGE}_{WIKI40B_VERSION}_test.parsed.jsonl.gz", "rt", encoding="utf-8") as f:
    for i, json_line in enumerate(f):
        article = json.loads(json_line)
        print(json.dumps(article, ensure_ascii=False, indent=4))
        print("------------------------------------------------")
        print()
        
        if i >= 2:
            break

{
    "version_id": "1848243370795951995",
    "wikidata_id": "Q11331136",
    "text": "\n_START_ARTICLE_\nビートたけしの教科書に載らない日本人の謎\n_START_SECTION_\n概要\n_START_PARAGRAPH_\n「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。_NEWLINE_新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。",
    "normalized_title": "ビートたけしの教科書に載らない日本人の謎",
    "normalized_sections": [
        {
            "name": "概要",
            "paragraph": "「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。_NEWLINE_新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。"
        }
    ],
    "normalized_text": "_START_ARTICLE_\nビートたけしの教科書に載らない日本人の謎\n_START_SECTION_\n概要\n_START_PARAGRAPH_\n「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。_NEWLINE_新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。\n"
}
------------------------------------------------

{
    "version_id": "13993402720669107168",
    "wikidata_id": "Q17218581",
    "text": "\n_START_ARTICLE_\nゲオネットワークス\n_START_SECTION_\n概要\n_START_PARAGRAPH_\nライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。_NEWLINE_2005年に株式会